<a href="https://colab.research.google.com/github/venku1BM22CS404/3rd_year_project/blob/main/Mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas textblob googletrans==4.0.0-rc1

In [ ]:
!pip install pyspellchecker

In [ ]:
!pip install emoji

In [ ]:
import nltk

# Download the 'averaged_perceptron_tagger' dataset
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [145]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
import re,string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from spellchecker import SpellChecker
import emoji
import pandas as pd
from googletrans import Translator
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from nltk.corpus import wordnet
from sklearn.metrics import classification_report

#google translate
translator = Translator(service_urls =['translate.google.com'])

#pyspellchecker
spell = SpellChecker()


In [146]:
#label 0 means negative and 4 means positive
data = pd.read_csv("test_data.csv",skip_blank_lines=True,encoding = "latin")
data

,ï»¿Label,number,date,no_query,name,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
39996,4,1960186342,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Madelinedugganx,My GrandMa is making Dinenr with my Mum
39997,4,1960186409,Fri May 29 07:33:43 PDT 2009,NO_QUERY,OffRoad_Dude,Mid-morning snack time... A bowl of cheese noo...
39998,4,1960186429,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Falchion,@ShaDeLa same here say it like from the Termi...
39999,4,1960186445,Fri May 29 07:33:44 PDT 2009,NO_QUERY,jonasobsessedx,@DestinyHope92 im great thaanks wbuu?


In [147]:
l=[]
for i in data["ï»¿Label"]:
    if(i==0):
        l.append("negative")
    else:
        l.append("positive")
data['label']=l

In [148]:
#dropping unwanted columns
data=data.drop(columns=['number', 'date','name','no_query','ï»¿Label'])

In [149]:
data

,Tweet,label
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative
1,is upset that he can't update his Facebook by ...,negative
2,@Kenichan I dived many times for the ball. Man...,negative
3,my whole body feels itchy and like its on fire,negative
4,"@nationwideclass no, it's not behaving at all....",negative
...,...,...
39996,My GrandMa is making Dinenr with my Mum,positive
39997,Mid-morning snack time... A bowl of cheese noo...,positive
39998,@ShaDeLa same here say it like from the Termi...,positive
39999,@DestinyHope92 im great thaanks wbuu?,positive


In [ ]:
#removes all emojis
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [150]:
def contractions():

    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "ily":"I love you",
        "Ily":"I love you",
        "Ihy":"I hate you",
        "ihy":"I hate you",
        "imy":"I miss you",
        "Imy":"I miss you",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "im":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks",
        "shit":"bad",
        "tmr":"tomorrow",
        "tmrw":"tomorrow",
        "u":"you",
        "ur":"your",
        "k":"okay",
        "ok":"okay",
        "da":"the",
        "tom":"tomorrow",
        "Tom":"tomorrow",
        "v'll":"we will",
        "wassup":"what is up with you",
        "waddup":"what is up with you",
        "yo":"greet",
        "hey":"greet",
        "lol":"laugh",
        "lmao":"laugh",
        "Lmao":"laugh",
        "rofl":"laugh",
        "y":"why",
        "wut":"what",
        "wat":"what",
        "stfu":"angry",
        "wtf":"angry",
        "ya":"yes",
        "yeah":"yes",
        "ummmm":"confused",
        "ummm":"confused",
        "umm":"confused",
        "hmmm":"confused",
        "i'm":"I am",
        "awww":"amazement",
        "Awww":"amazement",
        "aww":"amazement",
        "Aww":"amazement",
        "can't":"cannot",
        "Can't":"cannot",
        "CAN'T":"cannot",
        "awe":"amazement",
        "Awe":"amazement",
        "ugh":"sad",
        "ughh":"sad",
        "Ugh":"sad",
        "Ughh":"sad",
        "UGHH":"sad",
        "ughhhh":"sad",
        "ughhh":"sad"
        }



In [151]:
def emoticons():

    return {
        ":)":"smiley",
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":((((":"sad",
        ":(((":"sad",
        ":((":"sad",
        ":(":"sad",
        ":/":"sad",
        ":///":"sad",
        ":////":"sad",
        "://///":"sad",
        "://":"sad",
        ":///////":"sad",
        ":////":"sad",
        "-_-":"angry",
        ":|":"normal",
        ";)":"playful",
        ";D":"playful",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }




In [152]:
def removal_of_noise(sent):
    clean_sent=[]
    temp_st=""
    list_sent=sent.split(" ")
    c=0
    d=contractions()
    emoji=emoticons()
    for word in list_sent:
        #removal of url
        word = re.sub(r"http\S+", "", word)
        word = re.sub(r"[www.][a-zA-Z0-9_]+[.com]", "", word)
        #removal of account handles '@'
        word = re.sub("(@[A-Za-z0-9_]+)","", word)

        #replacing emoticons with their respective words
        if(word in emoji.keys()):
            word=emoji[word]
        #replacing short form words with their full form
        if(word.lower() in d.keys()):
            word=d[word.lower()]
        if(c==0):
            temp_st=word
        else:
            temp_st=temp_st+" "+word
        c=c+1
    sent=temp_st
    stop_words = set(stopwords.words('english'))
    stop_words.add('is')
    stop_words.remove('not')
    for word in word_tokenize(sent):
        if(word.lower() not in stop_words and word.lower() not in string.punctuation and word!="'" and word!='"' ):
            #print(word)
            word=spell.correction(word.lower())
            word=re.sub("[0-9]+","",word)
            word=re.sub("[.]+"," ",word)
            word=re.sub("[-]+"," ",word)
            word=re.sub("[_]+"," ",word)
            word = re.sub("~"," ", word)
            if(len(word)!=1):
                clean_sent.append(word.lower())
    cleaned_st=""
    for i in clean_sent:
        cleaned_st=cleaned_st+" "+i
    #print(cleaned_st)
    return lemmatization(cleaned_st)




In [153]:
def lemmatization(sent):
    lemmatize=WordNetLemmatizer()
    sentence_after_lemmatization=[]
    for word,tag in pos_tag(word_tokenize(sent)):
        if(tag[0:2]=="NN"):
            pos='n'
        elif(tag[0:2]=="VB"):
            pos='v'
        else:
            pos='a'
        lem=lemmatize.lemmatize(word,pos)
        sentence_after_lemmatization.append(lem)
    st=""
    for i in sentence_after_lemmatization:
        if(i!="be" and i!="is" and len(i)!=1):
            st=st+" "+i
    #print("lemi",st)
    c=0
    list_text=st.split()
    flag=0
    new_st=""
    for i in list_text:
        temp=i
        if(flag==1):
            flag=0
            continue
        if(i=="not" and (c+1)<len(list_text)):
            for syn in wordnet.synsets(list_text[c+1]):
                antonyms=[]
                for l in syn.lemmas():
                    #print(l)
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                        #print(antonyms)
                        temp=antonyms[0]
                        flag=1
                        break
                if(flag==1):
                    break
        new_st=new_st+" "+temp
        c+=1
    #print(new_st)
    return new_st



In [ ]:
#nltk module to get the sentiment polarity
def sentiment_analysis(sent):
        sid = SentimentIntensityAnalyzer()
        #print("-------------------------------------")
        print(sent)
        #print("-------------------------------------")
        ss = sid.polarity_scores(sent)
        x=ss['pos']
        y=ss['neg']
        print(x-y)
        print("-------------------------------------")
        return x-y



In [ ]:
def start(text):
    #removes html tags
    text =BeautifulSoup(text).get_text()
    text =text.replace("’","'")
    new_text=sent_tokenize(text)
    #print((new_text))
    result=0
    new_str=""
    #removing emojis
    for i in new_text:
        j=deEmojify(i)
        res=removal_of_noise(j)
        new_str=new_str+" "+res
    return new_str


In [ ]:
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def start(text):
    # Add debugging to print out the input text
    print("Input text:", text)

    # Check if the input is a string
    if not isinstance(text, str):
        print("Input text is not a string.")
        return ""

    #removes html tags
    text = BeautifulSoup(text).get_text()
    text = text.replace("’", "'")
    new_text = sent_tokenize(text)
    result = 0
    new_str = ""

    # Add more debugging to print out intermediate steps
    print("After removing HTML tags:", text)

    #removing emojis
    for i in new_text:
        j = deEmojify(i)
        res = removal_of_noise(j)
        new_str = new_str + " " + res

    print("After removing emojis and noise:", new_str)

    # Return the preprocessed text as 'x'
    x = new_str
    return x


In [154]:
with open('cleaned_tweet.txt', 'w') as f:
    for item in clean_list:
        f.write("%s\n" % item)

NameError: name 'clean_list' is not defined

In [ ]:
# Read from file cleaned tweets and store in a cleaned tweets column in the DataFrame
filename = "cleaned_tweet.txt"
with open(filename) as f:
    lines = f.read().splitlines()

# Check the length of lines read from the file
print("Number of lines read from file:", len(lines))

# Check the length of the DataFrame data
print("Number of rows in DataFrame data:", len(data))

# If the lengths match, assign the lines to the "cleaned_tweets" column
if len(lines) == len(data):
    data["cleaned_tweets"] = lines
    print("Successfully assigned values to 'cleaned_tweets' column.")
else:
    print("Lengths do not match. Investigate further.")

# Print the first few rows of the DataFrame to verify the new column
print(data.head())


Number of lines read from file: 0
Number of rows in DataFrame data: 40001
Lengths do not match. Investigate further.
                                               Tweet     label
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  negative
1  is upset that he can't update his Facebook by ...  negative
2  @Kenichan I dived many times for the ball. Man...  negative
3    my whole body feels itchy and like its on fire   negative
4  @nationwideclass no, it's not behaving at all....  negative


In [ ]:
#reading the adjective file
filename = "english-adjectives.txt"
with open(filename) as f:
    lines = f.read().splitlines()
lines
adjectives=lines

In [ ]:
#All adjectives words in the file
all_words=[]
negative=["not"]
for i in data["Tweet"]:
    for word in word_tokenize(i):
        if(word in adjectives or word in negative):
        #if(word in adjectives ):
            all_words.append(word)

len(all_words)

55287

In [ ]:
print(data.columns)



Index(['Tweet', 'label'], dtype='object')


In [ ]:
#creating a frequency distribution of each adjectives.
import nltk
BagOfWords = nltk.FreqDist(all_words)
BagOfWords
len(BagOfWords)

972

In [ ]:
# listing the  5000 most frequent words
word_features = list(BagOfWords.keys())[:5000]
len(word_features)
#word_features

972

In [ ]:
#after preprocessing data
data


,Tweet,label
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative
1,is upset that he can't update his Facebook by ...,negative
2,@Kenichan I dived many times for the ball. Man...,negative
3,my whole body feels itchy and like its on fire,negative
4,"@nationwideclass no, it's not behaving at all....",negative
...,...,...
39996,My GrandMa is making Dinenr with my Mum,positive
39997,Mid-morning snack time... A bowl of cheese noo...,positive
39998,@ShaDeLa same here say it like from the Termi...,positive
39999,@DestinyHope92 im great thaanks wbuu?,positive


In [ ]:
#assigning feature for each row in clean_tweets
new_list=[]
for i in data["Tweet"]:
    st=""
    for j in i.split():
        if(j in word_features):
            st=st+" "+j
    new_list.append(st)

data["cleaned_tweets"]=new_list

In [ ]:
data

,Tweet,label,cleaned_tweets
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative,
1,is upset that he can't update his Facebook by ...,negative,upset that
2,@Kenichan I dived many times for the ball. Man...,negative,
3,my whole body feels itchy and like its on fire,negative,whole itchy
4,"@nationwideclass no, it's not behaving at all....",negative,not all
...,...,...,...
39996,My GrandMa is making Dinenr with my Mum,positive,
39997,Mid-morning snack time... A bowl of cheese noo...,positive,
39998,@ShaDeLa same here say it like from the Termi...,positive,same
39999,@DestinyHope92 im great thaanks wbuu?,positive,great


In [ ]:
#Spliting into test data and train data
y=data["label"]
x=data.drop('label',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15)

In [ ]:
x_train.shape

(34000, 2)

In [156]:
import pandas as pd

# Ensure x_train and x_test are DataFrames
assert isinstance(x_train, pd.DataFrame), "x_train is not a DataFrame"
assert isinstance(x_test, pd.DataFrame), "x_test is not a DataFrame"

# Initialize X_train and X_test
X_train = pd.DataFrame(columns=['Tweet', 'cleaned_tweets'])
X_test = pd.DataFrame(columns=['Tweet', 'cleaned_tweets'])

# Append x_train and x_test to X_train and X_test
X_train = pd.concat([X_train, x_train], ignore_index=True)
X_test = pd.concat([X_test, x_test], ignore_index=True)

# Append labels to Y_train and Y_test
Y_train = y_train.copy()
Y_test = y_test.copy()

print("X_train:", X_train.head())
print("X_test:", X_test.head())
print("Y_train:", Y_train[:5])
print("Y_test:", Y_test[:5])


X_train:                                                Tweet  \
0                      folding washing - lots of it    
1                                    i have the flu    
2  Just saw Star Trek and remembered all the time...   
3  @hannahindie I am procrastinating on that pape...   
4  Listening to an old interview with JTG (John T...   

                        cleaned_tweets  
0                                       
1                                       
2   all used which love that more this  
3             that all other that that  
4                                  old  
X_test:                                                Tweet  cleaned_tweets
0  Has been looking at roman stuff, unfortunately...   unfortunately
1                                  Where's my mail?                 
2  Bed time now. Stuart's off to Wembley tomorrow...       luck blue
3  @melissamoog I am dying to visit! Unfortunatel...             not
4  @Lt_Algonquin @Lt_Algonquin Goodnight fair pri...   

In [158]:
# Splitting into train sets for training
training_set = []
for tweet, label in zip(X_train["cleaned_tweets"], Y_train):
    training_set.append((tweet.split(), label))

def list_to_dict(words_list):
    return dict([(word, True) for word in words_list])

# Format training set
training_set_formatted = [(list_to_dict(element[0]), element[1]) for element in training_set]

# Print or use the formatted training set
print(training_set_formatted)


[({}, 'negative'), ({}, 'negative'), ({'all': True, 'used': True, 'which': True, 'love': True, 'that': True, 'more': True, 'this': True}, 'positive'), ({'that': True, 'all': True, 'other': True}, 'negative'), ({'old': True}, 'negative'), ({'yesterday': True}, 'negative'), ({}, 'positive'), ({'love': True}, 'positive'), ({'back': True}, 'negative'), ({'too': True}, 'negative'), ({'that': True}, 'negative'), ({'extremely': True, 'fact': True, 'that': True, 'all': True}, 'negative'), ({'old': True, 'not': True, 'never': True}, 'negative'), ({'all': True, 'close': True}, 'positive'), ({'better': True}, 'positive'), ({'not': True, 'all': True}, 'negative'), ({}, 'negative'), ({'ill': True}, 'positive'), ({'honestly': True, 'all': True, 'same': True}, 'positive'), ({}, 'negative'), ({'true': True}, 'negative'), ({'very': True}, 'positive'), ({'actually': True}, 'positive'), ({'not': True, 'all': True}, 'negative'), ({}, 'positive'), ({}, 'positive'), ({}, 'negative'), ({'high': True}, 'posit

In [160]:
# Splitting into test sets for testing
test_set = []
for tweet, label in zip(X_test["cleaned_tweets"], Y_test):
    test_set.append((tweet.split(), label))

def list_to_dict(words_list):
    return dict([(word, True) for word in words_list])

# Format test set
test_set_formatted = [(list_to_dict(element[0]), element[1]) for element in test_set]

# Print or use the formatted test set
print(test_set_formatted)


[({'unfortunately': True}, 'negative'), ({}, 'negative'), ({'luck': True, 'blue': True}, 'negative'), ({'not': True}, 'negative'), ({'fair': True}, 'positive'), ({'big': True}, 'negative'), ({}, 'negative'), ({'best': True}, 'positive'), ({'very': True, 'pleased': True}, 'negative'), ({}, 'negative'), ({'never': True}, 'negative'), ({'front': True}, 'negative'), ({}, 'positive'), ({}, 'negative'), ({'some': True, 'crazy': True, 'tired': True}, 'negative'), ({}, 'positive'), ({}, 'positive'), ({'expensive': True, 'low': True}, 'positive'), ({}, 'positive'), ({}, 'positive'), ({'that': True, 'any': True}, 'positive'), ({'back': True, 'this': True, 'good': True}, 'negative'), ({'wrong': True}, 'negative'), ({}, 'positive'), ({'previous': True}, 'positive'), ({'sad': True}, 'negative'), ({}, 'positive'), ({'really': True, 'new': True, 'more': True}, 'negative'), ({'actually': True, 'our': True, 'that': True, 'some': True, 'this': True}, 'positive'), ({'soon': True}, 'negative'), ({'that': 

In [161]:
from sklearn.metrics import recall_score,precision_score
#making a list of classifiers with their names
classifiers=[]
#making a list of classifiers with their accuracy
accuracy=[]

In [162]:
#naive bayes classifier
classifier = nltk.NaiveBayesClassifier.train(training_set_formatted)

ground_truth = [r[1] for r in test_set_formatted]
preds = [classifier.classify(r[0]) for r in test_set_formatted]
f1score=f1_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'micro')
recallscore=recall_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'macro')
precisionscore=precision_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'weighted')

#accuracy
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, test_set_formatted))*100)
# print("F1-score : ",100*f1score)
# print("Recall Score : ",100*recallscore)
# print("Precision Score : ",100*precisionscore)
print()

classifier.show_most_informative_features(15)

classifiers.append([classifier,"naive bayes classifier"])

accuracy.append([(nltk.classify.accuracy(classifier, test_set_formatted))*100,"NB"])

print("Original Naive Bayes\n")
target_names = [ 'positive','negative']
print(classification_report(Y_test, preds, target_names=target_names))

Original Naive Bayes Algo accuracy percent: 60.22329611731379

Most Informative Features
                   worst = True           negati : positi =     29.9 : 1.0
                    sick = True           negati : positi =     16.9 : 1.0
                  lonely = True           negati : positi =     15.1 : 1.0
                    poor = True           negati : positi =     14.1 : 1.0
                     sad = True           negati : positi =     13.3 : 1.0
                 welcome = True           positi : negati =     13.2 : 1.0
                positive = True           positi : negati =     12.2 : 1.0
                  broken = True           negati : positi =     11.1 : 1.0
                terrible = True           negati : positi =      8.8 : 1.0
                   proud = True           positi : negati =      8.6 : 1.0
                    lost = True           negati : positi =      8.5 : 1.0
                    loss = True           negati : positi =      8.3 : 1.0
           

In [163]:
#Multinomail naive bayes
MNB_clf = SklearnClassifier(MultinomialNB())
MNB_clf.train(training_set_formatted)
print("Multinomail naive bayes classifier accuracy percent:", (nltk.classify.accuracy(MNB_clf, test_set_formatted))*100)

ground_truth = [r[1] for r in test_set_formatted]
preds = [MNB_clf.classify(r[0]) for r in test_set_formatted]
f1score=f1_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'micro')
recallscore=recall_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'macro')
precisionscore=precision_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'weighted')

# print("F1-score : ",100*f1score)
# print("Recall Score : ",100*recallscore)
# print("Precision Score : ",100*precisionscore)

accuracy.append([(nltk.classify.accuracy(MNB_clf, test_set_formatted))*100,"MNB"])

classifiers.append([MNB_clf,"Multinomail naive bayes classifier"])

print("Multinomail naive bayes\n")
target_names = [ 'positive','negative']
print(classification_report(Y_test, preds, target_names=target_names))

Multinomail naive bayes classifier accuracy percent: 60.38993501083153
Multinomail naive bayes

              precision    recall  f1-score   support

    positive       0.70      0.40      0.51      3078
    negative       0.56      0.81      0.67      2923

    accuracy                           0.60      6001
   macro avg       0.63      0.61      0.59      6001
weighted avg       0.63      0.60      0.59      6001



In [164]:
#Bernouli naive bayes
BNB_clf = SklearnClassifier(BernoulliNB())
BNB_clf.train(training_set_formatted)
print("Bernoulli naive bayes classifier accuracy percent:", (nltk.classify.accuracy(BNB_clf, test_set_formatted))*100)

ground_truth = [r[1] for r in test_set_formatted]
preds = [BNB_clf.classify(r[0]) for r in test_set_formatted]
f1score=f1_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'micro')
recallscore=recall_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'macro')
precisionscore=precision_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'weighted')

# print("F1-score : ",100*f1score)
# print("Recall Score : ",100*recallscore)
# print("Precision Score : ",100*precisionscore)

accuracy.append([(nltk.classify.accuracy(BNB_clf, test_set_formatted))*100,"BNB"])

classifiers.append([BNB_clf,"Bernouli classifier"])

print("Bernouli naive bayes\n")
target_names = [ 'positive','negative']
print(classification_report(Y_test, preds, target_names=target_names))

Bernoulli naive bayes classifier accuracy percent: 60.606565572404605
Bernouli naive bayes

              precision    recall  f1-score   support

    positive       0.69      0.42      0.52      3078
    negative       0.57      0.80      0.66      2923

    accuracy                           0.61      6001
   macro avg       0.63      0.61      0.59      6001
weighted avg       0.63      0.61      0.59      6001



In [165]:
#Logistic regression
LogReg_clf = SklearnClassifier(LogisticRegression())
LogReg_clf.train(training_set_formatted)
print("Logistic Regression classifier accuracy percent:", (nltk.classify.accuracy(LogReg_clf, test_set_formatted))*100)

ground_truth = [r[1] for r in test_set_formatted]
preds = [LogReg_clf.classify(r[0]) for r in test_set_formatted]
f1score=f1_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'micro')
recallscore=recall_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'macro')
precisionscore=precision_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'weighted')

# print("F1-score : ",100*f1score)
# print("Recall Score : ",100*recallscore)
# print("Precision Score : ",100*precisionscore)

accuracy.append([(nltk.classify.accuracy(LogReg_clf, test_set_formatted))*100,"LogReg"])


classifiers.append([LogReg_clf,"Bernouli LogisticRegression_classifier"])

print("Logistic regression\n")
target_names = [ 'positive','negative']
print(classification_report(Y_test, preds, target_names=target_names))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression classifier accuracy percent: 60.706548908515245
Logistic regression

              precision    recall  f1-score   support

    positive       0.69      0.42      0.52      3078
    negative       0.57      0.80      0.67      2923

    accuracy                           0.61      6001
   macro avg       0.63      0.61      0.59      6001
weighted avg       0.63      0.61      0.59      6001



In [166]:
#Stochastic Gradient Descent classifier
SGD_clf = SklearnClassifier(SGDClassifier())
SGD_clf.train(training_set_formatted)
print("Stochastic Gradient Descent Classifier_classifier accuracy percent:", (nltk.classify.accuracy(SGD_clf, test_set_formatted))*100)

ground_truth = [r[1] for r in test_set_formatted]
preds = [SGD_clf.classify(r[0]) for r in test_set_formatted]
f1score=f1_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'micro')
recallscore=recall_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'macro')
precisionscore=precision_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'weighted')

# print("F1-score : ",100*f1score)
# print("Recall Score : ",100*recallscore)
# print("Precision Score : ",100*precisionscore)

accuracy.append([(nltk.classify.accuracy(SGD_clf, test_set_formatted))*100,"SGD"])


classifiers.append([SGD_clf,"SGD classifier"])

print("Stochastic Gradient Descent\n")
target_names = [ 'positive','negative']
print(classification_report(Y_test, preds, target_names=target_names))

Stochastic Gradient Descent Classifier_classifier accuracy percent: 59.62339610064989
Stochastic Gradient Descent

              precision    recall  f1-score   support

    positive       0.72      0.35      0.47      3078
    negative       0.56      0.85      0.67      2923

    accuracy                           0.60      6001
   macro avg       0.64      0.60      0.57      6001
weighted avg       0.64      0.60      0.57      6001



In [167]:
#Support vector classifier
SVC_clf = SklearnClassifier(SVC())
SVC_clf.train(training_set_formatted)
print("Support vector classifier accuracy percent:", (nltk.classify.accuracy(SVC_clf, test_set_formatted))*100)

ground_truth = [r[1] for r in test_set_formatted]
preds = [SVC_clf.classify(r[0]) for r in test_set_formatted]
f1score=f1_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'micro')
recallscore=recall_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'macro')
precisionscore=precision_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'weighted')

# print("F1-score : ",100*f1score)
# print("Recall Score : ",100*recallscore)
# print("Precision Score : ",100*precisionscore)

accuracy.append([(nltk.classify.accuracy(SVC_clf, test_set_formatted))*100,"SVC"])

classifiers.append([SVC_clf,"SVC classifier"])

print("Support vector classifier\n")
target_names = [ 'positive','negative']
print(classification_report(Y_test, preds, target_names=target_names))


Support vector classifier accuracy percent: 59.89001833027828
Support vector classifier

              precision    recall  f1-score   support

    positive       0.67      0.43      0.53      3078
    negative       0.56      0.77      0.65      2923

    accuracy                           0.60      6001
   macro avg       0.62      0.60      0.59      6001
weighted avg       0.62      0.60      0.59      6001



In [168]:
#Max Entropy classifier
from nltk.classify import  MaxentClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import f1_score

def max_ent(training_set_formatted):
    numIterations = 100
    algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
    classifier = nltk.MaxentClassifier.train(training_set_formatted, algorithm, max_iter=numIterations)
    classifier.show_most_informative_features(10)
    return classifier

maxent_classifier=max_ent(training_set_formatted)


ground_truth = [r[1] for r in test_set_formatted]

preds = [maxent_classifier.classify(r[0]) for r in test_set_formatted]

f1score=f1_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'micro')
recallscore=recall_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'macro')
precisionscore=precision_score(ground_truth, preds, labels = ['negative', 'positive'], average = 'weighted')

print("Accuracy : ",f1_score(ground_truth, preds, labels = ['positive', 'negative'], average = 'micro')*100)
# print("F1-score : ",100*f1score)
# print("Recall Score : ",100*recallscore)
# print("Precision Score : ",100*precisionscore)

accuracy.append([(nltk.classify.accuracy(SVC_clf, test_set_formatted))*100,"MaxEnt"])


classifiers.append([maxent_classifier,"Max Entropy classifier"])

print("Max Entropy\n")
target_names = [ 'positive','negative']
print(classification_report(Y_test, preds, target_names=target_names))


  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.502
             2          -0.69297        0.622
             3          -0.69280        0.623
             4          -0.69263        0.623
             5          -0.69246        0.623
             6          -0.69229        0.623
             7          -0.69212        0.623
             8          -0.69194        0.623
             9          -0.69177        0.623
            10          -0.69160        0.623
            11          -0.69143        0.623
            12          -0.69126        0.623
            13          -0.69109        0.623
            14          -0.69092        0.623
            15          -0.69075        0.623
            16          -0.69058        0.623
            17          -0.69042        0.623
            18          -0.69025        0.623
            19          -0.69008        0.623
 

In [169]:
from nltk.classify import ClassifierI
from statistics import mode

# Defininig the ensemble model class

class EnsembleClassifier(ClassifierI):

    def __init__(self, *classifiers):
        self._classifiers = classifiers

    # returns the classification based on majority of votes
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)


In [173]:
def hybrid(test_set_formatted):
    ensemble_clf = EnsembleClassifier(classifiers[0][0], classifiers[1][0], classifiers[2][0], classifiers[3][0],classifiers[4][0],classifiers[5][0],classifiers[6][0])

    # List of only feature dictionary from the featureset list of tuples
    feature_list = [f[0] for f in test_set_formatted]
    global c
    # Looping over each to classify each review
    ensemble_preds = [ensemble_clf.classify(features) for features in feature_list]

    # Initialize counter for correct predictions
    c = 0

    for i in range(len(ensemble_preds)):
        if ensemble_preds[i] == test_set_formatted[i][1]:  # Accessing label from test_set_formatted
            c += 1

    return ensemble_preds

# Call the hybrid function
c = 0
preds = hybrid(test_set_formatted)

# Extract ground truth labels from test_set_formatted
ground_truth = [r[1] for r in test_set_formatted]

# Calculate accuracy
accuracy = 100 * c / len(preds)
print("Accuracy of hybrid: ", accuracy)

# Print classification report
target_names = ['positive', 'negative']
print(classification_report(ground_truth, preds, target_names=target_names))


Accuracy of hybrid:  60.58990168305282
              precision    recall  f1-score   support

    positive       0.69      0.42      0.52      3078
    negative       0.57      0.80      0.66      2923

    accuracy                           0.61      6001
   macro avg       0.63      0.61      0.59      6001
weighted avg       0.63      0.61      0.59      6001



In [174]:
accuracy

60.58990168305282

In [177]:
def features(text):
    new_list=[]
    for i in text.split():
        if(i in adjectives):
            new_list.append(i)
    return new_list

In [178]:
def text_classify(text):
    cleaned_text=start(text)
    temp=features(cleaned_text)
    test_data=list_to_dict(temp)
    print(temp)
    print("Tweet given by user : ",text)
    for i in classifiers:
        print(i[1])
        determined_label=i[0].classify(test_data)
        print("This Tweet is ",determined_label)
        print("------------------------------")
    c=0
    print("Hybrid model")
    testset_data=[]
    testset_data.append([test_data,""])
    lab=hybrid(testset_data)
    print("This Tweet is ",lab[0])

In [179]:
 nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [180]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [181]:
from nltk.corpus import twitter_samples

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
x=negative_tweets[:1000]
st=""
for i in x:
    st=st+" "+i
st

' hopeless for tmr :( Everything in the kids section of IKEA is so cute. Shame I\'m nearly 19 in 2 months :( @Hegelbon That heart sliding into the waste basket. :( “@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too Dang starting next week I have "work" :( oh god, my babies\' faces :( https://t.co/9fcwGvaki0 @RileyMcDonough make me smile :(( @f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln why?:("@tahuodyy: sialan:( https://t.co/Hv1i0xcrL2" Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz I have a really good m&amp;g idea but I\'m never going to meet them :((( @Rampageinthebox mare ivan :( @SophiaMascardo happy trip, keep safe. see you soon :* :( I\'m so tired hahahah :( @GrumpyCockney With knee replacements they get you up &amp; about the same day. :-(   Ouch. relate to the "sweet n\' sour" kind of "bi-polar

In [182]:

text_classify(st)


Input text:  hopeless for tmr :( Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :( @Hegelbon That heart sliding into the waste basket. :( “@ketchBurning: I hate Japanese call him "bani" :( :(”

Me too Dang starting next week I have "work" :( oh god, my babies' faces :( https://t.co/9fcwGvaki0 @RileyMcDonough make me smile :(( @f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln why?:("@tahuodyy: sialan:( https://t.co/Hv1i0xcrL2" Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz I have a really good m&amp;g idea but I'm never going to meet them :((( @Rampageinthebox mare ivan :( @SophiaMascardo happy trip, keep safe. see you soon :* :( I'm so tired hahahah :( @GrumpyCockney With knee replacements they get you up &amp; about the same day. :-(   Ouch. relate to the "sweet n' sour" kind of "bi-p

TypeError: expected string or bytes-like object

In [183]:
#input from the user which will be used to classify
def hinglish(input_text):
    translator = Translator(service_urls=['translate.google.co.in'])
    x=translator.translate(input_text,src="hi",dest="en")
    text_classify(x.text)

In [184]:
#input from the user which will be used to classify
from textblob import TextBlob

def hinglish2(input_text):
    l=input_text.split()
    st=""
    for i in l:
        word=TextBlob(i)
        if(word.detect_language()=="hi"):
            translator = Translator(service_urls=['translate.google.co.in'])
            x=translator.translate(i,src="hi",dest="en")
            st=st+" "+x.text
        else:
            st=st+" "+i
    text_classify(st)

In [185]:
def func(input_text):
    l=input_text.split()
    flag=0
    for i in l:
        k=len(i)
        if(k<3):
            flag=1
            hinglish(input_text)
    if(not(flag)):
        hinglish2(input_text)


In [203]:
func("tum bekar hu")

Input text: You're useless
After removing HTML tags: You're useless
After removing emojis and noise:   useless
['useless']
Tweet given by user :  You're useless
naive bayes classifier
This Tweet is  positive
------------------------------
Multinomail naive bayes classifier
This Tweet is  positive
------------------------------
Bernouli classifier
This Tweet is  positive
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  positive
------------------------------
SGD classifier
This Tweet is  positive
------------------------------
SVC classifier
This Tweet is  positive
------------------------------
Max Entropy classifier
This Tweet is  positive
------------------------------
Hybrid model
This Tweet is  positive


In [204]:
func("tum log kharab ho")

Input text: You guys are bad
After removing HTML tags: You guys are bad
After removing emojis and noise:   guy bad
['bad']
Tweet given by user :  You guys are bad
naive bayes classifier
This Tweet is  negative
------------------------------
Multinomail naive bayes classifier
This Tweet is  negative
------------------------------
Bernouli classifier
This Tweet is  negative
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  negative
------------------------------
SGD classifier
This Tweet is  negative
------------------------------
SVC classifier
This Tweet is  negative
------------------------------
Max Entropy classifier
This Tweet is  negative
------------------------------
Hybrid model
This Tweet is  negative


In [201]:
func("arrey waah! I'm very proud of you")

Input text: Arrey waah!I'm very proud of you
After removing HTML tags: Arrey waah!I'm very proud of you
After removing emojis and noise:   array wash proud
['proud']
Tweet given by user :  Arrey waah!I'm very proud of you
naive bayes classifier
This Tweet is  positive
------------------------------
Multinomail naive bayes classifier
This Tweet is  positive
------------------------------
Bernouli classifier
This Tweet is  positive
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  positive
------------------------------
SGD classifier
This Tweet is  positive
------------------------------
SVC classifier
This Tweet is  positive
------------------------------
Max Entropy classifier
This Tweet is  positive
------------------------------
Hybrid model
This Tweet is  positive


In [205]:
func("tum log pagal ho")

Input text: You guys are crazy
After removing HTML tags: You guys are crazy
After removing emojis and noise:   guy crazy
['crazy']
Tweet given by user :  You guys are crazy
naive bayes classifier
This Tweet is  positive
------------------------------
Multinomail naive bayes classifier
This Tweet is  positive
------------------------------
Bernouli classifier
This Tweet is  positive
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  positive
------------------------------
SGD classifier
This Tweet is  positive
------------------------------
SVC classifier
This Tweet is  negative
------------------------------
Max Entropy classifier
This Tweet is  positive
------------------------------
Hybrid model
This Tweet is  positive


In [213]:
func("tu bikhari ho")

Input text: You are scattered
After removing HTML tags: You are scattered
After removing emojis and noise:   scatter
[]
Tweet given by user :  You are scattered
naive bayes classifier
This Tweet is  positive
------------------------------
Multinomail naive bayes classifier
This Tweet is  positive
------------------------------
Bernouli classifier
This Tweet is  positive
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  positive
------------------------------
SGD classifier
This Tweet is  positive
------------------------------
SVC classifier
This Tweet is  positive
------------------------------
Max Entropy classifier
This Tweet is  positive
------------------------------
Hybrid model
This Tweet is  positive
Input text: You are scattered
After removing HTML tags: You are scattered
After removing emojis and noise:   scatter
[]
Tweet given by user :  You are scattered
naive bayes classifier
This Tweet is  positive
------------------------------
Multinom

In [214]:
func("tum pagal ho")

Input text: You are crazy
After removing HTML tags: You are crazy
After removing emojis and noise:   crazy
['crazy']
Tweet given by user :  You are crazy
naive bayes classifier
This Tweet is  positive
------------------------------
Multinomail naive bayes classifier
This Tweet is  positive
------------------------------
Bernouli classifier
This Tweet is  positive
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  positive
------------------------------
SGD classifier
This Tweet is  positive
------------------------------
SVC classifier
This Tweet is  negative
------------------------------
Max Entropy classifier
This Tweet is  positive
------------------------------
Hybrid model
This Tweet is  positive


In [215]:
func("tum kharab ho")

Input text: You are bad
After removing HTML tags: You are bad
After removing emojis and noise:   bad
['bad']
Tweet given by user :  You are bad
naive bayes classifier
This Tweet is  negative
------------------------------
Multinomail naive bayes classifier
This Tweet is  negative
------------------------------
Bernouli classifier
This Tweet is  negative
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  negative
------------------------------
SGD classifier
This Tweet is  negative
------------------------------
SVC classifier
This Tweet is  negative
------------------------------
Max Entropy classifier
This Tweet is  negative
------------------------------
Hybrid model
This Tweet is  negative


In [193]:
func("tu accha nahi hye")

Input text: You are not good
After removing HTML tags: You are not good
After removing emojis and noise:   evil
['evil']
Tweet given by user :  You are not good
naive bayes classifier
This Tweet is  negative
------------------------------
Multinomail naive bayes classifier
This Tweet is  negative
------------------------------
Bernouli classifier
This Tweet is  negative
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  negative
------------------------------
SGD classifier
This Tweet is  negative
------------------------------
SVC classifier
This Tweet is  negative
------------------------------
Max Entropy classifier
This Tweet is  negative
------------------------------
Hybrid model
This Tweet is  negative


In [194]:
func("tum log pagal aur kharab ho")

Input text: You guys are crazy and bad
After removing HTML tags: You guys are crazy and bad
After removing emojis and noise:   guy crazy bad
['crazy', 'bad']
Tweet given by user :  You guys are crazy and bad
naive bayes classifier
This Tweet is  negative
------------------------------
Multinomail naive bayes classifier
This Tweet is  negative
------------------------------
Bernouli classifier
This Tweet is  negative
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  negative
------------------------------
SGD classifier
This Tweet is  negative
------------------------------
SVC classifier
This Tweet is  negative
------------------------------
Max Entropy classifier
This Tweet is  negative
------------------------------
Hybrid model
This Tweet is  negative


In [195]:
func("tum faltu ho")

Input text: You are stupid
After removing HTML tags: You are stupid
After removing emojis and noise:   stupid
['stupid']
Tweet given by user :  You are stupid
naive bayes classifier
This Tweet is  negative
------------------------------
Multinomail naive bayes classifier
This Tweet is  negative
------------------------------
Bernouli classifier
This Tweet is  negative
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  negative
------------------------------
SGD classifier
This Tweet is  negative
------------------------------
SVC classifier
This Tweet is  negative
------------------------------
Max Entropy classifier
This Tweet is  negative
------------------------------
Hybrid model
This Tweet is  negative


In [199]:
func("Tu kuch fayde ka nahi ho")

Input text: You are not of some benefit
After removing HTML tags: You are not of some benefit
After removing emojis and noise:   not benefit
['not']
Tweet given by user :  You are not of some benefit
naive bayes classifier
This Tweet is  negative
------------------------------
Multinomail naive bayes classifier
This Tweet is  negative
------------------------------
Bernouli classifier
This Tweet is  negative
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  negative
------------------------------
SGD classifier
This Tweet is  negative
------------------------------
SVC classifier
This Tweet is  negative
------------------------------
Max Entropy classifier
This Tweet is  negative
------------------------------
Hybrid model
This Tweet is  negative
Input text: You are not of some benefit
After removing HTML tags: You are not of some benefit
After removing emojis and noise:   not benefit
['not']
Tweet given by user :  You are not of some benefit
naive ba

In [200]:
func("tu diwani hye")

Input text: tu diwani hai
After removing HTML tags: tu diwani hai
After removing emojis and noise:   to diwan have
[]
Tweet given by user :  tu diwani hai
naive bayes classifier
This Tweet is  positive
------------------------------
Multinomail naive bayes classifier
This Tweet is  positive
------------------------------
Bernouli classifier
This Tweet is  positive
------------------------------
Bernouli LogisticRegression_classifier
This Tweet is  positive
------------------------------
SGD classifier
This Tweet is  positive
------------------------------
SVC classifier
This Tweet is  positive
------------------------------
Max Entropy classifier
This Tweet is  positive
------------------------------
Hybrid model
This Tweet is  positive
